Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse

In [2]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [3]:
num_points = [100, 1_000, 5_000]
num_sets = num_points
density = [.3, .7]

x = make_set_covering_problem(num_points[0], num_sets[0], density[0])
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: True


In [4]:
from random import random, choice, randint, sample
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
from copy import copy

import numpy as np

class SingleStateSearch:

    def __init__(self, num_points, num_sets, density, early_stop = 50, tabu = False, simulated_annealing = False, iterated_local_search = False, tweak_type = 'random_hillclimber', **kwargs):
        self.sets = self.make_set_covering_problem(num_points, num_sets, density)
        self.num_sets = num_sets
        self.call_to_fitness = 0
        self.call_to_fitness_for_best = 0
        self.early_stop = early_stop
        seed(num_points*2654435761+num_sets+density)
        self.tweaks = {'random_hillclimber': self.random_hillclimber, 'steepest_ascent': self.steepest_ascent}
        self.tweak = self.tweaks[tweak_type]
        self.tabu_list = []
        self.tabu = tabu
        self.simulated_annealing = simulated_annealing
        self.iterated_local_search = iterated_local_search
        if simulated_annealing:
            self.temperature = kwargs['temperature']
        if iterated_local_search:
            self.countdown = kwargs['countdown']
        if tweak_type == 'steepest_ascent':
            if 'sample_size' in kwargs:
                self.sample_size = kwargs['sample_size']
            else:
                self.sample_size = 1

    def make_set_covering_problem(self, num_points, num_sets, density):
        return tuple(np.array([random() < density for _ in range(num_points)]) for _ in range(num_sets))

    def fitness(self, state, add_count = True):
        if add_count:
            self.call_to_fitness += 1
        cost = sum(state)
        valid = np.sum(
            reduce(
                np.logical_or,
                [self.sets[i] for i, t in enumerate(state) if t],
                np.array([False for _ in range(self.num_sets)]),
            )
        )
        return valid, -cost

    def random_hillclimber(self, state):
        new_state = copy(state)
        index = randint(0, self.num_sets - 1)
        new_state[index] = not new_state[index]
        return new_state
    
    def steepest_ascent(self, state):
        try:
            if isinstance(self.sample_size, float):
                indeces = sample(range(self.num_sets), int(self.sample_size*self.num_sets))
            else:
                indeces = sample(range(self.num_sets), self.sample_size)
        except TypeError:
            raise(f'sample_size must be int or float')

        best_state = state
        best_fitness = self.fitness(best_state)
        for index in indeces:
            new_state = copy(state)
            new_state[index] = not new_state[index]
            new_fitness = self.fitness(new_state)
            if best_fitness < new_fitness:
                best_state = new_state
                best_fitness = new_fitness
        return best_state

    def solve(self):
        current_state = [choice([False, False, False, False, False, False]) for _ in range(self.num_sets)]
        self.best_sol = current_state
        num_steps_for_best = 0
        self.best_fitness = self.fitness(current_state)
        current_fitness = self.best_fitness
        it_no_improve = 0

        for step in range(10_000):
            it_no_improve += 1
            new_state = self.tweak(current_state)
            if self.tabu:
                if new_state in self.tabu_list:
                    continue 
                else:
                    self.tabu_list.append(new_state)
            new_fitness = self.fitness(new_state)
            if new_fitness >= current_fitness:
                current_state = new_state
                current_fitness = new_fitness
                if new_fitness > self.best_fitness:
                    it_no_improve = 0
                    self.call_to_fitness_for_best = self.call_to_fitness
                    self.best_fitness = new_fitness
                    self.best_sol = new_state
                    num_steps_for_best = step
                # print(self.fitness(current_state, add_count=False), step)
            else:
                if self.simulated_annealing:
                    p = np.exp(-(sum(current_fitness)-sum(new_fitness))/self.temperature)
                    if random() < p:
                        current_state = new_state
                        current_fitness = new_fitness
                        # print(self.fitness(current_state, add_count=False), step)
            if self.iterated_local_search:
                if step % self.countdown == 0:
                    current_state = self.best_sol
                    current_fitness = self.best_fitness
            if it_no_improve > self.early_stop:
                break
        
        return self.call_to_fitness_for_best, self.best_fitness, num_steps_for_best


In [7]:
results = []

for d in [0.3, 0.7]:
    for n in [100, 1000, 5000]:
        for tabu in [True, False]:
            for tweak_type in ['steepest_ascent', 'random_hillclimber']:
                for iterated_local_search in [True, False]:
                    res = SingleStateSearch(n, n, d, tweak_type = tweak_type, tabu = tabu, simulated_annealing=False, iterated_local_search=iterated_local_search, temperature = 0.1, sample_size=10, countdown = 20).solve()
                    arg_dict = {'density': d, 'num_sets': n, 'tabu': tabu, 'tweak_type': tweak_type, 'iterated_local_search': iterated_local_search}
                    results.append((res, arg_dict))


results

[((120, (100, -7), 9),
  {'density': 0.3,
   'num_sets': 100,
   'tabu': True,
   'tweak_type': 'steepest_ascent',
   'iterated_local_search': True}),
 ((120, (100, -7), 9),
  {'density': 0.3,
   'num_sets': 100,
   'tabu': True,
   'tweak_type': 'steepest_ascent',
   'iterated_local_search': False}),
 ((17, (100, -13), 15),
  {'density': 0.3,
   'num_sets': 100,
   'tabu': True,
   'tweak_type': 'random_hillclimber',
   'iterated_local_search': True}),
 ((76, (100, -9), 84),
  {'density': 0.3,
   'num_sets': 100,
   'tabu': True,
   'tweak_type': 'random_hillclimber',
   'iterated_local_search': False}),
 ((85, (100, -7), 6),
  {'density': 0.3,
   'num_sets': 100,
   'tabu': False,
   'tweak_type': 'steepest_ascent',
   'iterated_local_search': True}),
 ((97, (100, -8), 7),
  {'density': 0.3,
   'num_sets': 100,
   'tabu': False,
   'tweak_type': 'steepest_ascent',
   'iterated_local_search': False}),
 ((111, (100, -10), 109),
  {'density': 0.3,
   'num_sets': 100,
   'tabu': False,
 

In [9]:
new_results = []

for result in results:
    new_dict = result[1]
    new_dict['fitness calls'] = result[0][0]
    new_dict['fitness score'] = result[0][1]
    new_results.append(new_dict)

In [12]:
import pandas as pd

pd.DataFrame(new_results).to_csv('./results.csv')